In [ ]:
# Datasets had to be updated so each set had consistent variables. Also, some sample data had been left out of the
# metadata files, which needed to be added.

## Most of the datasets required the same corrections: fixing the tax table column names, adding "PyOM" and "PyOM.pH" 
##columns, fixing spelling mistakes, fixing/correcting sample data column names.


In [3]:
# Loading R packages
library(reshape)
library(ggplot2)
library(phyloseq)
library(plyr)
library(dplyr)
library(plotly)
library(wesanderson)

# Wu

In [79]:
ps_wu = import_biom("Wu_OTU_table/feature-table-metaD-tax_json.biom" , parseFunction = parse_taxonomy_default)

In [80]:
##Fixing the tax_table object to "clean-up" the column names

x = data.frame(tax_table(ps_wu))
# Making a dummy variable to store the taxonomy data

colnames(x) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks

x$Domain = gsub("D_0__", "", as.character(x$Domain))
x$Phylum = gsub("D_1__", "", as.character(x$Phylum))
x$Class = gsub("D_2__", "", as.character(x$Class))
x$Order = gsub("D_3__", "", as.character(x$Order))
x$Family = gsub("D_4__", "", as.character(x$Family))
x$Genus = gsub("D_5__", "", as.character(x$Genus))
x$Species = gsub("D_6__", "", as.character(x$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x=tax_table(as.matrix(x,dimnames=list(row.names(x),colnames(x))))
# Turning it into a taxonomy table, while saving the rownames and column names
tax_table(ps_wu)=x
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_wu))
# Check for success

In [81]:
#no soil type was provided in the publication; we looked at the paper to obtain a rough estimate on where the soil was 
#collected- the soil type was sourced from the FAO/UNESCO Soil Map of the World

##Adding the soil-type to the Wu sample data
sample_data(ps_wu)[sample_data(ps_wu)$Author=="Wu",]$SoilType = "Gleysol"

In [82]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"
sample_data(ps_wu)$Treatment = revalue(sample_data(ps_wu)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_wu)$SoilType = revalue(sample_data(ps_wu)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_wu)$Feedstock = revalue(sample_data(ps_wu)$Feedstock, c("none"="NA"))
sample_data(ps_wu)$Habitat = revalue(sample_data(ps_wu)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_wu)$CurrentLandUse = revalue(sample_data(ps_wu)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_wu)$PyOM.Temp = revalue(sample_data(ps_wu)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_wu)$Depth = revalue(sample_data(ps_wu)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unammended, unamended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, minimim tillage agriculture field, field
The following `from` values were not present in `x`: N/A, 0
The following `from` values were not present in `x`: 15cm


In [6]:
#Fixing the titles so PyOM.Temp and t.per.ha.PyOM are consistent among datasets
names(sample_data(ps_wu))[names(sample_data(ps_wu)) == "PyOM-Temp"] <- "PyOM.Temp"
names(sample_data(ps_wu))[names(sample_data(ps_wu)) == "t.per.hakg.PyOM"] <- "t.per.ha.PyOM"

[1] "GeographicalLocation"  "PyOM.Temp"             "Sampling_Depth_cm"    
 [4] "CurrentLandUse"        "FwdBarcode"            "NucleicAcidExtraction"
 [7] "Feedstock"             "%C"                    "ForwardPrimerName"    
[10] "RevBarcode"            "t.per.ha.PyOM"         "Silt"                 
[13] "Sand"                  "RevPrimerName"         "Clay"                 
[16] "Treatment"             "BioSample"             "Organism"             
[19] "SoilType"              "Habitat"               "Incubation_Time_Weeks"
[22] "BioProject"            "Crop"                  "ReversePrimerSequence"
[25] "CollectionYear"        "TargetGene"            "SequencingMethod"     
[28] "ForwardPrimerSequence" "Author"                "SRA"                  
[31] "pH"                    "Region"                "PyOM"                 
[34] "PyOM.pH"

[1] "GeographicalLocation"  "PyOM.Temp"             "Sampling_Depth_cm"    
 [4] "CurrentLandUse"        "FwdBarcode"            "NucleicAcidExtraction"
 [7] "Feedstock"             "%C"                    "ForwardPrimerName"    
[10] "RevBarcode"            "t.per.ha.PyOM"         "Silt"                 
[13] "Sand"                  "RevPrimerName"         "Clay"                 
[16] "Treatment"             "BioSample"             "Organism"             
[19] "SoilType"              "Habitat"               "Incubation_Time_Weeks"
[22] "BioProject"            "Crop"                  "ReversePrimerSequence"
[25] "CollectionYear"        "TargetGene"            "SequencingMethod"     
[28] "ForwardPrimerSequence" "Author"                "SRA"                  
[31] "pH"                    "Region"                "PyOM"                 
[34] "PyOM.pH"

In [4]:
#Create new variables to identify whether char, OM, or nothing was added to the sample
CharOnlyAdded = c('1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA")
OMAdded = c('Composted biochar and straw + soil','Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [5]:
#Create new column "PyOM" to identify whether char was added or not using the above variables
sample_data(ps_wu)$PyOM = ifelse(sample_data(ps_wu)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_wu)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_wu)$Treatment %in% OMAdded, "OM", NA)))

[1] "GeographicalLocation"  "PyOM.Temp"             "Sampling_Depth_cm"    
 [4] "CurrentLandUse"        "FwdBarcode"            "NucleicAcidExtraction"
 [7] "Feedstock"             "%C"                    "ForwardPrimerName"    
[10] "RevBarcode"            "t.per.ha.PyOM"         "Silt"                 
[13] "Sand"                  "RevPrimerName"         "Clay"                 
[16] "Treatment"             "BioSample"             "Organism"             
[19] "SoilType"              "Habitat"               "Incubation_Time_Weeks"
[22] "BioProject"            "Crop"                  "ReversePrimerSequence"
[25] "CollectionYear"        "TargetGene"            "SequencingMethod"     
[28] "ForwardPrimerSequence" "Author"                "SRA"                  
[31] "pH"                    "Region"                "PyOM"                 
[34] "PyOM.pH"

[1] "Control" "OM"      "PyOM"    "OM"      "PyOM"    "OM"

In [37]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data
sample_data(ps_wu)$PyOM.pH = ifelse(sample_data(ps_wu)$Treatment %in% "Biochar + soil", 9.98 ,
                                    ifelse(sample_data(ps_wu)$Treatment %in% "Composted biochar + soil", 7.13, NA))

[1]   NA   NA 9.98   NA 7.13   NA

In [38]:
#Save as an R object so that the phyloseq object can be called in future processing
saveRDS(ps_wu, "wu.ps")

# Imparato

In [80]:

ps_imparato = import_biom("Imparato_OTU_table/feature-table-metaD-tax_json.biom" , parseFunction = parse_taxonomy_default)

In [81]:
##Fixing the tax_table object to "clean-up" the column names
x.imparato = data.frame(tax_table(ps_imparato))
colnames(x.imparato) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks

x.imparato$Domain = gsub("D_0__", "", as.character(x.imparato$Domain))
x.imparato$Phylum = gsub("D_1__", "", as.character(x.imparato$Phylum))
x.imparato$Class = gsub("D_2__", "", as.character(x.imparato$Class))
x.imparato$Order = gsub("D_3__", "", as.character(x.imparato$Order))
x.imparato$Family = gsub("D_4__", "", as.character(x.imparato$Family))
x.imparato$Genus = gsub("D_5__", "", as.character(x.imparato$Genus))
x.imparato$Species = gsub("D_6__", "", as.character(x.imparato$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x.imparato=tax_table(as.matrix(x.imparato,dimnames=list(row.names(x.imparato),colnames(x.imparato))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_imparato)=x.imparato
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_imparato))
# Check for success

In [82]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

sample_data(ps_imparato)$Treatment = revalue(sample_data(ps_imparato)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_imparato)$SoilType = revalue(sample_data(ps_imparato)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_imparato)$Feedstock = revalue(sample_data(ps_imparato)$Feedstock, c("none"="NA"))
sample_data(ps_imparato)$Habitat = revalue(sample_data(ps_imparato)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_imparato)$CurrentLandUse = revalue(sample_data(ps_imparato)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_imparato)$PyOM.Temp = revalue(sample_data(ps_imparato)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_imparato)$Depth = revalue(sample_data(ps_imparato)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unammended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, field
The following `from` values were not present in `x`: N/A, 0


In [102]:
#Fixing the titles so PyOM.Temp and t.per.ha.PyOM are consistent among datasets

names(sample_data(ps_imparato))[names(sample_data(ps_imparato)) == "PyOM-Temp"] <- "PyOM.Temp"
names(sample_data(ps_imparato))[names(sample_data(ps_imparato)) == "t.per.hakg.PyOM"] <- "t.per.ha.PyOM"
colnames(sample_data(ps_imparato))

[1] "Feedstock"             "Clay"                  "t.per.hakg.PyOM"      
 [4] "NucleicAcidExtraction" "pH"                    "%C"                   
 [7] "ForwardPrimerName"     "Crop"                  "ForwardPrimerSequence"
[10] "TargetGene"            "Incubation_Time_Weeks" "Region"               
[13] "Silt"                  "BioProject"            "%SOM"                 
[16] "CurrentLandUse"        "Organism"              "Author"               
[19] "PyOM.Temp"             "FwdBarcode"            "RevBarcode"           
[22] "BioSample"             "SoilType"              "Habitat"              
[25] "RevPrimerName"         "Depth"                 "SequencingMethod"     
[28] "Treatment"             "Sand"                  "GeographicalLocation" 
[31] "SRA"                   "CollectionYear"        "ReversePrimerSequence"
[34] "PyOM"

[1] "Feedstock"             "Clay"                  "t.per.ha.PyOM"        
 [4] "NucleicAcidExtraction" "pH"                    "%C"                   
 [7] "ForwardPrimerName"     "Crop"                  "ForwardPrimerSequence"
[10] "TargetGene"            "Incubation_Time_Weeks" "Region"               
[13] "Silt"                  "BioProject"            "%SOM"                 
[16] "CurrentLandUse"        "Organism"              "Author"               
[19] "PyOM.Temp"             "FwdBarcode"            "RevBarcode"           
[22] "BioSample"             "SoilType"              "Habitat"              
[25] "RevPrimerName"         "Depth"                 "SequencingMethod"     
[28] "Treatment"             "Sand"                  "GeographicalLocation" 
[31] "SRA"                   "CollectionYear"        "ReversePrimerSequence"
[34] "PyOM"

In [83]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA")
OMAdded = c('Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [84]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_imparato)$PyOM = ifelse(sample_data(ps_imparato)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_imparato)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_imparato)$Treatment %in% OMAdded, "OM", NA)))

In [39]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data

sample_data(ps_imparato)$PyOM.pH = ifelse(sample_data(ps_imparato)$PyOM %in% "PyOM", 11.6 , NA)


[1] 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6 11.6
[16] 11.6 11.6 11.6   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA   NA
[31]   NA   NA   NA   NA   NA   NA

In [40]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_imparato, "imparato.ps")

# Dai 2017

In [117]:
ps_dai2017 = import_biom("Dai2017_OTU_table/feature-table-metaD-tax_json.biom", parseFunction=parse_taxonomy_default)

In [118]:
##Fixing the tax_table object to "clean-up" the column names
x.dai2017 = data.frame(tax_table(ps_dai2017))

colnames(x.dai2017) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks

x.dai2017$Domain = gsub("D_0__", "", as.character(x.dai2017$Domain))
x.dai2017$Phylum = gsub("D_1__", "", as.character(x.dai2017$Phylum))
x.dai2017$Class = gsub("D_2__", "", as.character(x.dai2017$Class))
x.dai2017$Order = gsub("D_3__", "", as.character(x.dai2017$Order))
x.dai2017$Family = gsub("D_4__", "", as.character(x.dai2017$Family))
x.dai2017$Genus = gsub("D_5__", "", as.character(x.dai2017$Genus))
x.dai2017$Species = gsub("D_6__", "", as.character(x.dai2017$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x.dai2017=tax_table(as.matrix(x.dai2017,dimnames=list(row.names(x.dai2017),colnames(x.dai2017))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_dai2017)=x.dai2017
# Reassigning the taxonomy table in ps_xxx to the new modified one

(tax_table(ps_dai2017))
# Check for success

In [119]:
#Author did not include what treatment corresponded with which sample, so hypothetical matches were recorded based on NMDS 
#of Bray-Curtis and some sample ordering logic

##Create a matrix to match samples with likely treatments

Dai2017Matrix = c('SRR3681435-X',"Argiustoll","Manure", "Ar300", "PyOM",300,
'SRR3681436-X',"Argiustoll", "Manure","Ar300", "PyOM", 300,
'SRR3681437-X', "Argiustoll","Manure","Ar300", "PyOM", 300,
'SRR3681438-X', "Argiustoll","Manure","Ar700", "PyOM", 700,
'SRR3681439-X', "Argiustoll","Manure","Ar700", "PyOM", 700,
'SRR3681440-X', "Argiustoll","Manure","Ar700", "PyOM", 700,
'SRR3681441-X', "Argiustoll",NA,"ArCK", "Control", NA,
'SRR3681442-X', "Argiustoll",NA,"ArCK", "Control", NA,
'SRR3681471-X', "Argiustoll",NA,"ArCK", "Control", NA,
'SRR3681472-X',"PyOM", "Manure","PyAr300", NA, 300,
'SRR3681473-X', "PyOM","Manure","PyAr300", NA, 300,
'SRR3681474-X',"PyOM", "Manure","PyAr300", NA, 300,
'SRR3681475-X', "PyOM","Manure","PyAr700", NA, 700,
'SRR3681476-X', "PyOM","Manure","PyAr700", NA, 700,
'SRR3681477-X',"PyOM", "Manure","PyAr700", NA, 700,
'SRR3681478-X',"Psammaquent", "Manure","Ps300", "PyOM", 300,
'SRR3681479-X', "Psammaquent","Manure","Ps300", "PyOM", 300,
'SRR3681480-X', "Psammaquent","Manure","Ps300", "PyOM", 300,
'SRR3681481-X',"Psammaquent", "Manure","Ps700", "PyOM", 700,
'SRR3681482-X',"Psammaquent", "Manure","Ps700", "PyOM", 700,
'SRR3681483-X',"Psammaquent", "Manure","Ps700", "PyOM", 700,
'SRR3681484-X', "Psammaquent",NA,"PsCK", "Control", NA,
'SRR3681485-X', "Psammaquent",NA,"PsCK", "Control", NA,
'SRR3681486-X', "Psammaquent",NA,"PsCK", "Control", NA,
'SRR3681488-X', "PyOM","Manure","PyPs300", NA, 300,
'SRR3681490-X', "PyOM","Manure","PyPs300", NA, 300,
'SRR3681491-X', "PyOM","Manure","PyPs300", NA, 300,
'SRR3681492-X', "PyOM","Manure","PyPs700", NA, 700,
'SRR3681494-X', "PyOM","Manure","PyPs700", NA, 700,
'SRR3681496-X', "PyOM","Manure","PyPs700", NA, 700)

Dai2017Matrix = t(matrix(Dai2017Matrix,nrow = 6))
colnames(Dai2017Matrix) = c("SampleID","SoilType","Feedstock","Treatment","PyOM","PyOM.Temp")
row.names(Dai2017Matrix) = data.frame(Dai2017Matrix)$SampleID
Dai2017Matrix = data.frame(Dai2017Matrix[,2:6])
#check for success
head(Dai2017Matrix)

,SoilType,Feedstock,Treatment,PyOM,PyOM.Temp
SRR3681435-X,Argiustoll,Manure,Ar300,PyOM,300
SRR3681436-X,Argiustoll,Manure,Ar300,PyOM,300
SRR3681437-X,Argiustoll,Manure,Ar300,PyOM,300
SRR3681438-X,Argiustoll,Manure,Ar700,PyOM,700
SRR3681439-X,Argiustoll,Manure,Ar700,PyOM,700
SRR3681440-X,Argiustoll,Manure,Ar700,PyOM,700


In [120]:
# Fixing the Dai2017 sample data
## Paste information from matrix created above into the ps
sample_data(ps_dai2017)[row.names(sample_data(ps_dai2017)) %in% row.names(Dai2017Matrix),]$SoilType = paste(Dai2017Matrix$SoilType)
sample_data(ps_dai2017)[row.names(sample_data(ps_dai2017)) %in% row.names(Dai2017Matrix),]$Feedstock = paste(Dai2017Matrix$Feedstock)
sample_data(ps_dai2017)[row.names(sample_data(ps_dai2017)) %in% row.names(Dai2017Matrix),]$Treatment = paste(Dai2017Matrix$Treatment)
sample_data(ps_dai2017)[row.names(sample_data(ps_dai2017)) %in% row.names(Dai2017Matrix),]$PyOM = paste(Dai2017Matrix$PyOM)
sample_data(ps_dai2017)[row.names(sample_data(ps_dai2017)) %in% row.names(Dai2017Matrix),]$PyOM.Temp = paste(Dai2017Matrix$PyOM.Temp)

Warning message in `[<-.data.frame`(x3, i, , ..., value = value):
“provided 32 variables to replace 31 variables”

In [122]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

sample_data(ps_dai2017)$Treatment = revalue(sample_data(ps_dai2017)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_dai2017)$SoilType = revalue(sample_data(ps_dai2017)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_dai2017)$Feedstock = revalue(sample_data(ps_dai2017)$Feedstock, c("none"="NA"))
sample_data(ps_dai2017)$Habitat = revalue(sample_data(ps_dai2017)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_dai2017)$CurrentLandUse = revalue(sample_data(ps_dai2017)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_dai2017)$PyOM.Temp = revalue(sample_data(ps_dai2017)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_dai2017)$Depth = revalue(sample_data(ps_dai2017)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unammended, unamended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, minimim tillage agriculture field, field
The following `from` values were not present in `x`: N/A, 0
The following `from` values were not present in `x`: 15cm


In [121]:
#Fixing the titles so PyOM.Temp and g.per.kg.PyOM are consistent among datasets

names(sample_data(ps_dai2017))[names(sample_data(ps_dai2017)) == "PyOM-Temp"] <- "PyOM.Temp"
names(sample_data(ps_dai2017))[names(sample_data(ps_dai2017)) == "g-per-kg-PyOM"] <- "g.per.kg.PyOM"

# Changing pH to NA because pH data is the baseline soil pH, not the "true" pH at the time of
# soil collection (treatment + soil)
sample_data(ps_dai2017)$pH = NA


[1] "TargetGene"            "CurrentLandUse"        "NucleicAcidExtraction"
 [4] "SequencingMethod"      "Incubation_Time_Weeks" "SRA"                  
 [7] "Region"                "ForwardPrimerName"     "g.per.kg.PyOM"        
[10] "ForwardPrimerSequence" "pH"                    "Treatment"            
[13] "Author"                "RevPrimerName"         "Clay"                 
[16] "PyOM.Temp"             "FwdBarcode"            "Habitat"              
[19] "Sampling_Depth_cm"     "Sand"                  "CollectionYear"       
[22] "Feedstock"             "BioSample"             "RevBarcode"           
[25] "Organism"              "BioProject"            "SoilType"             
[28] "ReversePrimerSequence" "Silt"                  "GeographicalLocation" 
[31] "Crop"

[1] "300" "300" "300" "700" "700" "700" "NA"  "NA"  "NA"  "300" "300" "300"
[13] "700" "700" "700" "300" "300" "300" "700" "700" "700" "NA"  "NA"  "NA" 
[25] "300" "300" "300" "700" "700" "700"

[1] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[26] NA NA NA NA NA

In [123]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM', "Ar300", "Ar700", "Ps300", "Ps700")
NothingAdded = c("Control","Unamended","NA", "ArCK", "PsCK" )
OMAdded = c('Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [124]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_dai2017)$PyOM = ifelse(sample_data(ps_dai2017)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_dai2017)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_dai2017)$Treatment %in% OMAdded, "OM", NA)))

In [125]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data

sample_data(ps_dai2017)$PyOM.pH = ifelse(sample_data(ps_dai2017)$Treatment %in% "Ar300", 7.2,
                                         ifelse(sample_data(ps_dai2017)$Treatment %in% "Ar700", 9.6,
                                                ifelse(sample_data(ps_dai2017)$Treatment %in% "PyAr300", 7.2,
                                                       ifelse(sample_data(ps_dai2017)$Treatment %in% "PyAr700", 9.6,
                                                              ifelse(sample_data(ps_dai2017)$Treatment %in% "Ps300", 7.2,
                                                                     ifelse(sample_data(ps_dai2017)$Treatment %in% "Ps700", 9.6,
                                                                            ifelse(sample_data(ps_dai2017)$Treatment %in% "PyPs300", 7.2,
                                                                                   ifelse(sample_data(ps_dai2017)$Treatment %in% "PyPs700", 9.6, NA))))))))

sample_data(ps_dai2017)$PyOM.pH

[1] 7.2 7.2 7.2 9.6 9.6 9.6  NA  NA  NA 7.2 7.2 7.2 9.6 9.6 9.6 7.2 7.2 7.2 9.6
[20] 9.6 9.6  NA  NA  NA 7.2 7.2 7.2 9.6 9.6 9.6

In [128]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_dai2017, "dai2017.ps")

# Dai 2016

In [134]:
ps_dai2016 = import_biom("Dai2016_OTU_table/feature-table-metaD-tax_json.biom", parseFunction=parse_taxonomy_default)

In [135]:
##Fixing the tax_table object to "clean-up" the column names

x.dai2016 = data.frame(tax_table(ps_dai2016))

colnames(x.dai2016) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks

x.dai2016$Domain = gsub("D_0__", "", as.character(x.dai2016$Domain))
x.dai2016$Phylum = gsub("D_1__", "", as.character(x.dai2016$Phylum))
x.dai2016$Class = gsub("D_2__", "", as.character(x.dai2016$Class))
x.dai2016$Order = gsub("D_3__", "", as.character(x.dai2016$Order))
x.dai2016$Family = gsub("D_4__", "", as.character(x.dai2016$Family))
x.dai2016$Genus = gsub("D_5__", "", as.character(x.dai2016$Genus))
x.dai2016$Species = gsub("D_6__", "", as.character(x.dai2016$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x.dai2016=tax_table(as.matrix(x.dai2016,dimnames=list(row.names(x.dai2016),colnames(x.dai2016))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_dai2016)=x.dai2016
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_dai2016))
# Check for success

,Domain,Phylum,Class,Order,Family,Genus,Species
50b1f8869b00b7101fc4b6a263f1e3a2,Bacteria,Firmicutes,Bacilli,Bacillales,NA,NA,NA
938cda0a26fe3f948909700a8bd71a61,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Methylophilaceae,NA,NA
270ae734802f97123fcafc2d1d2eff61,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Rhodocyclaceae,NA,NA
88a09b8bc83628992633f4a96ca83bc5,Bacteria,Actinobacteria,Actinobacteria,Micrococcales,Intrasporangiaceae,Terrabacter,Ambiguous_taxa
cf5974e3786fe60b4beb8bcae9a14957,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Burkholderiaceae,Ramlibacter,NA
7a44aec2778c5cd208f236de47c87e5f,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Burkholderiaceae,Ramlibacter,NA


In [136]:
#Taking out a sample that had no sequences 
ps_dai2016 = prune_samples(sample_names(ps_dai2016)!= "SRR3452692-X", ps_dai2016)

In [10]:
#Author didn't include treatment for each sample, so below are the hypothetical matches, based on relative 
#abundances in samples
# Indicators: CK soils have higher acidos and lower bacteroidetes
# Indicators: Ma char has higher Firmicutes than St char
# BuCK has higher BetaProteo than RhCK
# Should be able to figure it out from that.
# Sample IDs do seem to be in order of treatment (group in 3's sort of)

Dai2016Matrix = c('SRR3466117-X', "Inceptisol", NA, "BuCK", "Control", 5.33,
                  'SRR3466120-X', "Inceptisol", NA, "BuCK", "Control", 5.33,
                  'SRR3466121-X', "Inceptisol", NA, "BuCK", "Control", 5.33,
                  'SRR3466122-X', "Inceptisol", "Manure", "BuMa", "PyOM", 6.64,
                  'SRR3466123-X', "Inceptisol", "Manure", "BuMa", "PyOM",  6.64,
                  'SRR3466124-X', "Inceptisol", "Manure", "BuMa", "PyOM",  6.64,
                  'SRR3466125-X', "Inceptisol", "Stover", "BuSt", "PyOM", 5.99,
                  'SRR3466126-X', "Inceptisol", "Stover", "BuSt", "PyOM", 5.99,
                  'SRR3466127-X', "Inceptisol", "Stover", "BuSt", "PyOM", 5.99,
                  'SRR3466128-X', "Inceptisol", NA, "RhCK", "Control", 5.13,
                  'SRR3466129-X', "Inceptisol", NA, "RhCK", "Control", 5.13,
                  'SRR3466130-X', "Inceptisol", NA, "RhCK", "Control", 5.13,
                  'SRR3466131-X', "Inceptisol", "Manure", "RhMa", "PyOM", 6.94,
                  'SRR3466134-X', "Inceptisol", "Manure", "RhMa", "PyOM", 6.94,
                  'SRR3466136-X', "Inceptisol", "Manure", "RhMa", "PyOM", 6.94,
                  'SRR3466137-X', "Inceptisol", "Stover", "RhSt", "PyOM", 6.11,
                  'SRR3466138-X', "Inceptisol", "Stover", "RhSt", "PyOM", 6.11,
                  'SRR3466140-X', "Inceptisol", "Stover", "RhSt", "PyOM", 6.11)
Dai2016Matrix = t(matrix(Dai2016Matrix,nrow = 6))
colnames(Dai2016Matrix) = c("SampleID","SoilType","Feedstock","Treatment","PyOM","pH")
row.names(Dai2016Matrix) = data.frame(Dai2016Matrix)$SampleID
Dai2016Matrix = data.frame(Dai2016Matrix[,2:6])

[1] "SoilType"  "Feedstock" "Treatment" "PyOM"      "pH"

In [138]:
# Fixing the Dai2016 sample data
## Paste information from matrix created above into the ps

sample_data(ps_dai2016)[row.names(sample_data(ps_dai2016)) %in% row.names(Dai2016Matrix),]$SoilType = paste(Dai2016Matrix$SoilType)
sample_data(ps_dai2016)[row.names(sample_data(ps_dai2016)) %in% row.names(Dai2016Matrix),]$Feedstock = paste(Dai2016Matrix$Feedstock)
sample_data(ps_dai2016)[row.names(sample_data(ps_dai2016)) %in% row.names(Dai2016Matrix),]$Treatment = paste(Dai2016Matrix$Treatment)
sample_data(ps_dai2016)[row.names(sample_data(ps_dai2016)) %in% row.names(Dai2016Matrix),]$PyOM = paste(Dai2016Matrix$PyOM)
sample_data(ps_dai2016)[row.names(sample_data(ps_dai2016)) %in% row.names(Dai2016Matrix),]$pH = paste(Dai2016Matrix$pH)

Warning message in `[<-.data.frame`(x3, i, , ..., value = value):
“provided 31 variables to replace 30 variables”

In [139]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

sample_data(ps_dai2016)$Treatment = revalue(sample_data(ps_dai2016)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_dai2016)$SoilType = revalue(sample_data(ps_dai2016)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_dai2016)$Feedstock = revalue(sample_data(ps_dai2016)$Feedstock, c("none"="NA"))
sample_data(ps_dai2016)$Habitat = revalue(sample_data(ps_dai2016)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_dai2016)$CurrentLandUse = revalue(sample_data(ps_dai2016)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_dai2016)$PyOM.Temp = revalue(sample_data(ps_dai2016)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_dai2016)$Depth = revalue(sample_data(ps_dai2016)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unammended, unamended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, minimim tillage agriculture field, field
The following `from` values were not present in `x`: N/A, 0
The following `from` values were not present in `x`: 15cm


In [140]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM', "BuMa", "BuSt", "RhMa", "RhSt")
NothingAdded = c("Control","Unamended","NA", "BuCK", "RhCK")
OMAdded = c('Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [141]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_dai2016)$PyOM = ifelse(sample_data(ps_dai2016)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_dai2016)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_dai2016)$Treatment %in% OMAdded, "OM", NA)))

In [45]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data

sample_data(ps_dai2016)$PyOM.pH = ifelse(sample_data(ps_dai2016)$Feedstock %in% "Manure", 9.55,
                                         ifelse(sample_data(ps_dai2016)$Feedstock %in% "Stover", 10.09, NA))

[1]    NA    NA    NA  9.55  9.55  9.55 10.09 10.09 10.09    NA    NA    NA
[13]  9.55  9.55  9.55 10.09 10.09 10.09

In [2]:
ps_dai2016 = readRDS (file = "dai2016.ps")

In [3]:
colnames(sample_data(ps_dai2016))
sample_data(ps_dai2016)$Incubation_Time_Weeks = 

[1] "Clay"                  "Feedstock"             "Organism"             
 [4] "SRA"                   "Region"                "BioProject"           
 [7] "FwdBarcode"            "ReversePrimerSequence" "CollectionYear"       
[10] "ForwardPrimerSequence" "ForwardPrimerName"     "RevPrimerName"        
[13] "CurrentLandUse"        "Sand"                  "Treatment"            
[16] "GeographicalLocation"  "Author"                "PyOM.Temp"            
[19] "TargetGene"            "RevBarcode"            "Habitat"              
[22] "g.per.kg.PyOM"         "pH"                    "Crop"                 
[25] "Sampling_Depth_cm"     "SequencingMethod"      "SoilType"             
[28] "BioSample"             "Silt"                  "NucleicAcidExtraction"
[31] "PyOM"                  "PyOM.pH"

In [46]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_dai2016, "dai2016.ps")

# Yao

In [103]:
ps_yao = import_biom("YaoBact_OTU_table/feature-table-metaD-tax_json.biom", parseFunction=parse_taxonomy_default)

In [104]:
##Fixing the tax_table object to "clean-up" the column names

x = data.frame(tax_table(ps_yao))

colnames(x) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks

x$Domain = gsub("D_0__", "", as.character(x$Domain))
x$Phylum = gsub("D_1__", "", as.character(x$Phylum))
x$Class = gsub("D_2__", "", as.character(x$Class))
x$Order = gsub("D_3__", "", as.character(x$Order))
x$Family = gsub("D_4__", "", as.character(x$Family))
x$Genus = gsub("D_5__", "", as.character(x$Genus))
x$Species = gsub("D_6__", "", as.character(x$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x=tax_table(as.matrix(x,dimnames=list(row.names(x),colnames(x))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_yao)=x
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_yao))
# Check for success

,Domain,Phylum,Class,Order,Family,Genus,Species
c2cdaf01de4b2ce2e02c78559acdff90,Bacteria,Actinobacteria,Actinobacteria,Micrococcales,Micrococcaceae,NA,NA
9b64bd7579ea761af3631fb70b05724d,Bacteria,Acidobacteria,Blastocatellia (Subgroup 4),Pyrinomonadales,Pyrinomonadaceae,RB41,NA
985d0b63e502e950e3d6a750178187f5,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Xanthobacteraceae,Bradyrhizobium,NA
75538196200bee33d344705bce5c6be0,Bacteria,Chloroflexi,Chloroflexia,Thermomicrobiales,JG30-KF-CM45,uncultured bacterium,uncultured bacterium
6b45812af17b96798fdf1fbb314fc383,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Nitrosomonadaceae,Ellin6067,uncultured bacterium
2ff7057427130406645db002539545b1,Bacteria,Acidobacteria,Subgroup 6,NA,NA,NA,NA


In [105]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

sample_data(ps_yao)$Treatment = revalue(sample_data(ps_yao)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_yao)$SoilType = revalue(sample_data(ps_yao)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_yao)$Feedstock = revalue(sample_data(ps_yao)$Feedstock, c("none"="NA"))
sample_data(ps_yao)$Habitat = revalue(sample_data(ps_yao)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_yao)$CurrentLandUse = revalue(sample_data(ps_yao)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_yao)$PyOM.Temp = revalue(sample_data(ps_yao)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_yao)$Depth = revalue(sample_data(ps_yao)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unammended, unamended
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, minimim tillage agriculture field, field
The following `from` values were not present in `x`: N/A, 0
The following `from` values were not present in `x`: 15cm


In [47]:
#Fixing the titles so PyOM.Temp and t.per.ha.PyOM are consistent among datasets

names(sample_data(ps_yao))[names(sample_data(ps_yao)) == "PyOM_Temp"] <- "PyOM.Temp"
names(sample_data(ps_yao))[names(sample_data(ps_yao)) == "t-per-ha-PyOM"] <- "t.per.ha.PyOM"

[1] "CollectionDate"        "TargetGene"            "Organism"             
 [4] "Sand"                  "Silt"                  "Clay"                 
 [7] "FwdBarcode"            "Region"                "RevBarcode"           
[10] "Treatment"             "SoilType"              "Sampling_Depth_cm"    
[13] "Habitat"               "SRA"                   "TotalC"               
[16] "t.per.ha.PyOM"         "SequencingMethod"      "ForwardPrimerSequence"
[19] "Feedstock"             "ForwardPrimerName"     "BioProject"           
[22] "pH"                    "NucleicAcidExtraction" "CurrentLandUse"       
[25] "RevPrimerName"         "PyOM.Temp"             "Crop"                 
[28] "GeographicalLocation"  "TotalN"                "ReversePrimerSequence"
[31] "Author"                "Incubation_Time_Weeks" "BioSample"            
[34] "PyOM"                  "PyOM.pH"

[1] "CollectionDate"        "TargetGene"            "Organism"             
 [4] "Sand"                  "Silt"                  "Clay"                 
 [7] "FwdBarcode"            "Region"                "RevBarcode"           
[10] "Treatment"             "SoilType"              "Sampling_Depth_cm"    
[13] "Habitat"               "SRA"                   "TotalC"               
[16] "t.per.ha.PyOM"         "SequencingMethod"      "ForwardPrimerSequence"
[19] "Feedstock"             "ForwardPrimerName"     "BioProject"           
[22] "pH"                    "NucleicAcidExtraction" "CurrentLandUse"       
[25] "RevPrimerName"         "PyOM.Temp"             "Crop"                 
[28] "GeographicalLocation"  "TotalN"                "ReversePrimerSequence"
[31] "Author"                "Incubation_Time_Weeks" "BioSample"            
[34] "PyOM"                  "PyOM.pH"

In [106]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA")
OMAdded = c('Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [107]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_yao)$PyOM = ifelse(sample_data(ps_yao)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_yao)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_yao)$Treatment %in% OMAdded, "OM", NA)))

In [49]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data

sample_data(ps_yao)$PyOM.pH = ifelse(sample_data(ps_yao)$Treatment %in% "biochar", 8.87, NA)

[1]   NA   NA   NA 8.87 8.87 8.87 8.87 8.87 8.87 8.87 8.87 8.87   NA   NA   NA
[16] 8.87 8.87 8.87 8.87 8.87 8.87 8.87 8.87 8.87   NA   NA   NA 8.87 8.87 8.87
[31] 8.87 8.87 8.87 8.87 8.87 8.87   NA   NA   NA 8.87 8.87 8.87 8.87 8.87 8.87
[46] 8.87 8.87 8.87

In [108]:
#make pH a numeric value

sample_data(ps_yao)$pH = as.numeric(sample_data(ps_yao)$pH)

In [50]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_yao, "yao.ps")

# Song

In [129]:
ps_song = import_biom("Song_OTU_table/feature-table-metaD-tax-assume_json.biom", parseFunction=parse_taxonomy_default)

In [135]:
##Fixing the tax_table object to "clean-up" the column names

x.song = data.frame(tax_table(ps_song))

colnames(x.song) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks

x.song$Domain = gsub("D_0__", "", as.character(x.song$Domain))
x.song$Phylum = gsub("D_1__", "", as.character(x.song$Phylum))
x.song$Class = gsub("D_2__", "", as.character(x.song$Class))
x.song$Order = gsub("D_3__", "", as.character(x.song$Order))
x.song$Family = gsub("D_4__", "", as.character(x.song$Family))
x.song$Genus = gsub("D_5__", "", as.character(x.song$Genus))
x.song$Species = gsub("D_6__", "", as.character(x.song$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x.song=tax_table(as.matrix(x.song,dimnames=list(row.names(x.song),colnames(x.song))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_song)=x.song
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_song))
# Check for success

In [132]:
#Create a matrix to use to add incubation time and pH to sample IDs
SongMatrix = c('SRR3984709-X',0, 7.5,
'SRR3984710-X',0, 7.5,
'SRR3984721-X',4, 7.5,
'SRR3984743-X',12, 7.5,
'SRR3984755-X',12, 7.5,
'SRR3984767-X',24, 7.5,
'SRR3984779-X',24, 7.5,
'SRR3984792-X',0, 7.4,
'SRR3984793-X',0, 7.4,
'SRR3984711-X',4, 7.4,
'SRR3984712-X',4, 7.4,
'SRR3984713-X',12, 7.4,
'SRR3984714-X',12, 7.4,
'SRR3984715-X',24, 7.4,
'SRR3984716-X',24, 7.4,
'SRR3984717-X',0, 7.5,
'SRR3984719-X',4, 7.5,
'SRR3984720-X',4, 7.5,
'SRR3984722-X',12, 7.5,
'SRR3984723-X',12, 7.5,
'SRR3984725-X',24, 7.5,
'SRR3984726-X',0, 7.4, 
'SRR3984727-X',0, 7.4,
'SRR3984728-X',4, 7.4,
'SRR3984730-X',12, 7.4,
'SRR3984731-X',12, 7.4,
'SRR3984733-X',24, 7.4,
'SRR3984734-X',24, 7.4,
'SRR3984735-X',0, 7.6,
'SRR3984736-X',0, 7.6,
'SRR3984737-X',4, 7.6,
'SRR3984738-X',4, 7.6,
'SRR3984740-X',12, 7.6,
'SRR3984741-X',24, 7.6,
'SRR3984742-X',24, 7.6)
SongMatrix = t(matrix(SongMatrix,nrow = 3))
colnames(SongMatrix) = c("SampleID","Incubation_Time_Weeks", "pH")
row.names(SongMatrix) = data.frame(SongMatrix)$SampleID
SongMatrix = data.frame(SongMatrix[,])
SongMatrix

,SampleID,Incubation_Time_Weeks,pH
SRR3984709-X,SRR3984709-X,0,7.5
SRR3984710-X,SRR3984710-X,0,7.5
SRR3984721-X,SRR3984721-X,4,7.5
SRR3984743-X,SRR3984743-X,12,7.5
SRR3984755-X,SRR3984755-X,12,7.5
SRR3984767-X,SRR3984767-X,24,7.5
SRR3984779-X,SRR3984779-X,24,7.5
SRR3984792-X,SRR3984792-X,0,7.4
SRR3984793-X,SRR3984793-X,0,7.4
SRR3984711-X,SRR3984711-X,4,7.4


In [133]:
# Paste incubation time from matrix created above into the ps
sample_data(ps_song)[row.names(sample_data(ps_song)) %in% row.names(SongMatrix),]$Incubation_Time_Weeks = paste(SongMatrix$Incubation_Time_Weeks)
sample_data(ps_song)$pH = sample_data(ps_song)[row.names(sample_data(ps_song)) %in% row.names(SongMatrix),]$pH = paste(SongMatrix$pH)

sample_data(ps_song)

,SequencingMethod,TargetGene,Habitat,NCBISampleID,ForwardPrimerName,Sampling_Depth_cm,ForwardPrimerSequence,g.per.kg.PyOM,CollectionYear,ReversePrimerSequence,⋯,X.SOM,Region,PyOM.Temp,pH,Incubation_Time_Weeks,GeographicalLocation,Treatment,Crop,Conc_PAH_ng_per_g,Author
SRR3984709-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,A01,338F,20,ACTCCTACGGGAGGCAGCA,0,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,0,7.4,0,China,control,,7581.61,Song
SRR3984710-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,A02,338F,20,ACTCCTACGGGAGGCAGCA,0,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,0,7.4,0,China,control,,7581.61,Song
SRR3984711-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,B11,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,300,7.4,4,China,1%BC300,,7581.61,Song
SRR3984712-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,B12,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,300,7.4,12,China,1%BC300,,7581.61,Song
SRR3984713-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,B31,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,300,7.4,12,China,1%BC300,,7581.61,Song
SRR3984714-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,B32,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,300,7.4,24,China,1%BC300,,7581.61,Song
SRR3984715-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,B61,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,300,7.4,24,China,1%BC300,,7581.61,Song
SRR3984716-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,B62,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,300,7.4,0,China,1%BC300,,7581.61,Song
SRR3984717-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,C01,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,600,7.4,0,China,1%BC600,,7581.61,Song
SRR3984719-X,Illumina MiSeq,16S rRNA,PAH-contaminated field,C11,338F,20,ACTCCTACGGGAGGCAGCA,10.1,,GGACTACHVGGGTWTCTAAT,⋯,2,V4-V5,600,7.4,4,China,1%BC600,,7581.61,Song


In [136]:
#no soil type was provided in the publication; we looked at the paper to obtain a rough estimate on where the soil was 
#collected- the soil type was sourced from the FAO/UNESCO Soil Map of the World
sample_data(ps_song)$SoilType = sample_data(ps_song)[sample_data(ps_song)$Author=="Song",]$SoilType = "Gleysol"

Warning message in `[<-.data.frame`(x3, i, , ..., value = value):
“provided 34 variables to replace 33 variables”

In [137]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

sample_data(ps_song)$Treatment = revalue(sample_data(ps_song)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_song)$SoilType = revalue(sample_data(ps_song)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_song)$Feedstock = revalue(sample_data(ps_song)$Feedstock, c("none"="NA"))
sample_data(ps_song)$Habitat = revalue(sample_data(ps_song)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_song)$CurrentLandUse = revalue(sample_data(ps_song)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_song)$PyOM.Temp = revalue(sample_data(ps_song)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_song)$Depth = revalue(sample_data(ps_song)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: unammended, unamended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, minimim tillage agriculture field
The following `from` values were not present in `x`: N/A
The following `from` values were not present in `x`: 15cm


In [130]:
#Fixing the titles so PyOM.Temp and t.per.ha.PyOM are consistent among datasets

names(sample_data(ps_song))[names(sample_data(ps_song)) == "PyOM-Temp"] <- "PyOM.Temp"
names(sample_data(ps_song))[names(sample_data(ps_song)) == "g-per-kg-PyOM"] <- "g.per.kg.PyOM"

[1] "SequencingMethod"      "TargetGene"            "Habitat"              
 [4] "NCBISampleID"          "ForwardPrimerName"     "Sampling_Depth_cm"    
 [7] "ForwardPrimerSequence" "g.per.kg.PyOM"         "CollectionYear"       
[10] "ReversePrimerSequence" "Silt"                  "BioSample"            
[13] "RevBarcode"            "RevPrimerName"         "Feedstock"            
[16] "Sand"                  "Organism"              "BioProject"           
[19] "Clay"                  "SRA"                   "FwdBarcode"           
[22] "NucleicAcidExtraction" "CurrentLandUse"        "%SOM"                 
[25] "Region"                "PyOM.Temp"             "pH"                   
[28] "Incubation_Time_Weeks" "GeographicalLocation"  "Treatment"            
[31] "Crop"                  "Conc_PAH_ng_per_g"     "Author"

[1] "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24"
[16] "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24" "24"
[31] "24" "24" "24" "24" "24"

In [138]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA")
OMAdded = c('Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [139]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_song)$PyOM = ifelse(sample_data(ps_song)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_song)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_song)$Treatment %in% OMAdded, "OM", NA)))

In [141]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data

sample_data(ps_song)$PyOM.pH = ifelse(sample_data(ps_song)$PyOM.Temp %in% "300", 7.2,
                                        ifelse(sample_data(ps_song)$PyOM.Temp %in% "600", 10.8 , NA))

[1]   NA   NA  7.2  7.2  7.2  7.2  7.2  7.2 10.8 10.8 10.8   NA 10.8 10.8 10.8
[16]  7.2  7.2  7.2  7.2  7.2  7.2  7.2 10.8 10.8 10.8 10.8 10.8 10.8 10.8   NA
[31]   NA   NA   NA  7.2  7.2

In [145]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_song, "song.ps")

# Whitman

In [65]:
ps_whitman = import_biom("Whitman_OTU_table/feature-table-metaD-tax_json-ncbiID.biom", parseFunction=parse_taxonomy_default)

In [66]:
#Prune out sample that didn't have any reads
ps_whitman= prune_samples(sample_names(ps_whitman)!= "SRR3305959-X", ps_whitman)

In [67]:
#some samples had duplicate sequencing files - the next few lines will be merging the duplicated based on sample name
ps.merged = merge_samples(ps_whitman,"NCBISampleID")

Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMeth

In [68]:
#create variable with all the column names
names=colnames(sample_data(ps_whitman))
names

[1] "ForwardPrimerName"     "SoilType"              "RevBarcode"           
 [4] "NucleicAcidExtraction" "Feedstock"             "GeographicalLocation" 
 [7] "ReversePrimerSequence" "pH"                    "RevPrimerName"        
[10] "TargetGene"            "X.N"                   "ForwardPrimerSequence"
[13] "Crop"                  "SequencingMethod"      "t.per.ha.PyOM"        
[16] "Author"                "Region"                "Organism"             
[19] "Habitat"               "CurrentLandUse"        "Silt"                 
[22] "Sand"                  "NCBISampleID"          "FwdBarcode"           
[25] "PyOM.Temp"             "Incubation_Time_Weeks" "Sampling_Depth_cm"    
[28] "SRA"                   "Clay"                  "Treatment"            
[31] "CollectionYear"        "BioProject"            "X.C"                  
[34] "BioSample"

In [69]:
notcommonnames = c('RevBarcode','FwdBarcode','SRA')
# Create list of sample data items that are not necessarily the same within the merged pairs
commonnames = names[(names %in% notcommonnames)]
# Pull out only the sample data categories that are the same
common_sample_data = sample_data(ps_whitman)[ , -which(names(sample_data(ps_whitman)) %in% commonnames)]
# Get the sample data for those common categories

In [70]:
sample_data(ps_whitman)$pH = as.numeric(sample_data(ps_whitman)$pH)

In [71]:
commonrownames=row.names(sample_data(ps.merged))
# Grab the rownames of the merged sample data - i.e., the new sample IDs
common_sample_data2 = common_sample_data[which(common_sample_data$NCBISampleID %in% commonrownames),]
# Pull out only one row for each sample ID
common_sample_data2 = common_sample_data2[!duplicated(common_sample_data2$NCBISampleID), ]
# Cut out all duplicates (from merging)
row.names(common_sample_data2) = common_sample_data2$NCBISampleID
# Rename rows with the new NCBI common sample ID
sample_data(ps.merged)=common_sample_data2
# Reassign sample data to the ps.merged object

# Renaming it all
ps_whitman=ps.merged
ps_whitman


phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 8628 taxa and 119 samples ]
sample_data() Sample Data:       [ 119 samples by 31 sample variables ]
tax_table()   Taxonomy Table:    [ 8628 taxa by 7 taxonomic ranks ]

In [72]:
##Fixing the tax_table object to "clean-up" the column names

x.whitman = data.frame(tax_table(ps_whitman))

colnames(x.whitman) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks

x.whitman$Domain = gsub("D_0__", "", as.character(x.whitman$Domain))
x.whitman$Phylum = gsub("D_1__", "", as.character(x.whitman$Phylum))
x.whitman$Class = gsub("D_2__", "", as.character(x.whitman$Class))
x.whitman$Order = gsub("D_3__", "", as.character(x.whitman$Order))
x.whitman$Family = gsub("D_4__", "", as.character(x.whitman$Family))
x.whitman$Genus = gsub("D_5__", "", as.character(x.whitman$Genus))
x.whitman$Species = gsub("D_6__", "", as.character(x.whitman$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x.whitman=tax_table(as.matrix(x.whitman,dimnames=list(row.names(x.whitman),colnames(x.whitman))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_whitman)=x.whitman
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_whitman))
# Check for success

,Domain,Phylum,Class,Order,Family,Genus,Species
224c444fe97e6aa6bc29426d54fe494b,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Xanthobacteraceae,Bradyrhizobium,NA
3d171376b6712ac4e9f7639c4bd5324f,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,uncultured Sphingomonadaceae bacterium
532bc584b044143342bf078fd5bfd49b,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,uncultured Sphingomonadaceae bacterium
d0bfc877c648962e3b7c31828dab4797,Bacteria,Actinobacteria,Actinobacteria,Micrococcales,Micrococcaceae,NA,NA
a8e90563afa8945d6bc7e9f008106ee3,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Burkholderiaceae,Massilia,Ambiguous_taxa
be23fcd748d2621432e14c0487c93e05,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Xanthobacteraceae,Bradyrhizobium,NA


In [74]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

#Note: in SoilType we changed "Mardin" to "Fragiudept" 

sample_data(ps_whitman)$Treatment = revalue(sample_data(ps_whitman)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_whitman)$SoilType = revalue(sample_data(ps_whitman)$SoilType, c("mollisol"="Mollisol" ,"Mardin" = "Fragiudept" ))
sample_data(ps_whitman)$Feedstock = revalue(sample_data(ps_whitman)$Feedstock, c("none"="NA"))
sample_data(ps_whitman)$Habitat = revalue(sample_data(ps_whitman)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_whitman)$CurrentLandUse = revalue(sample_data(ps_whitman)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_whitman)$PyOM.Temp = revalue(sample_data(ps_whitman)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_whitman)$Depth = revalue(sample_data(ps_whitman)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unamended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: minimim tillage agriculture field, field
The following `from` values were not present in `x`: N/A, 0
The following `from` values were not present in `x`: 15cm


In [76]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA")
OMAdded = c('Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [77]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_whitman)$PyOM = ifelse(sample_data(ps_whitman)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_whitman)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_whitman)$Treatment %in% OMAdded, "OM", NA)))

In [70]:
#Create new column "PyOM.pH" to identify the pH of the PyOM addition
#Fix pH from baseline soil pH to "true" pH after treatment additions
sample_data(ps_whitman)$PyOM.pH = ifelse(sample_data(ps_whitman)$PyOM %in% "PyOM", 10.0, NA)
sample_data(ps_whitman)$pH = ifelse(sample_data(ps_whitman)$PyOM %in% "PyOM", 6.75, 6)

[1] NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA
 [26] 10 NA 10 NA 10 NA 10 NA NA NA NA NA NA NA NA NA 10 NA 10 NA 10 NA 10 NA 10
 [51] NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA NA NA
 [76] NA NA NA NA NA NA 10 NA 10 NA 10 NA 10 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA
[101] 10 NA 10 NA 10 NA 10 NA 10 NA 10 NA NA NA NA NA NA NA NA

[1] 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00
 [16] 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75
 [31] 6.00 6.75 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.75 6.00 6.75 6.00
 [46] 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75
 [61] 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.00 6.00
 [76] 6.00 6.00 6.00 6.00 6.00 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.75 6.00
 [91] 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.75
[106] 6.00 6.75 6.00 6.75 6.00 6.75 6.00 6.00 6.00 6.00 6.00 6.00 6.00 6.00

In [73]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_whitman, "whitman.ps")

## Ye

In [2]:
ps_ye = import_biom("Ye_OTU_table-v3/feature-table-metaD-tax_json.biom", parseFunction=parse_taxonomy_default)

In [3]:
##Fixing the tax_table object to "clean-up" the column names

x = data.frame(tax_table(ps_ye))

colnames(x) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks
x$Domain = gsub("D_0__", "", as.character(x$Domain))
x$Phylum = gsub("D_1__", "", as.character(x$Phylum))
x$Class = gsub("D_2__", "", as.character(x$Class))
x$Order = gsub("D_3__", "", as.character(x$Order))
x$Family = gsub("D_4__", "", as.character(x$Family))
x$Genus = gsub("D_5__", "", as.character(x$Genus))
x$Species = gsub("D_6__", "", as.character(x$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x=tax_table(as.matrix(x,dimnames=list(row.names(x),colnames(x))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_ye)=x
# Reassigning the taxonomy table in ps_xxx to the new modified one

tax_table(ps_ye)
# Check for success

In [4]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

sample_data(ps_ye)$Treatment = revalue(sample_data(ps_ye)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_ye)$SoilType = revalue(sample_data(ps_ye)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_ye)$Feedstock = revalue(sample_data(ps_ye)$Feedstock, c("none"="NA"))
sample_data(ps_ye)$Habitat = revalue(sample_data(ps_ye)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_ye)$CurrentLandUse = revalue(sample_data(ps_ye)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_ye)$PyOM.Temp = revalue(sample_data(ps_ye)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_ye)$Depth = revalue(sample_data(ps_ye)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unammended, unamended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, minimim tillage agriculture field, field
The following `from` values were not present in `x`: N/A, 0
The following `from` values were not present in `x`: 15cm


In [6]:
#Fixing the titles so PyOM.Temp and t.per.ha.PyOM are consistent among datasets

names(sample_data(ps_ye))[names(sample_data(ps_ye)) == "PyOM-Temp"] <- "PyOM.Temp"
names(sample_data(ps_ye))[names(sample_data(ps_ye)) == "g.per.kg.PyOM"] <- "t.per.ha.PyOM"

[1] "RevPrimerName"         "Author"                "pH"                   
 [4] "Region"                "BioSample"             "Sampling_Depth_cm"    
 [7] "Avail_K"               "ForwardPrimerSequence" "TotalN"               
[10] "CurrentLandUse"        "TotalC"                "Organism"             
[13] "SequencingMethod"      "Habitat"               "Incubation_Time_Weeks"
[16] "Org_C"                 "GeographicalLocation"  "Clay"                 
[19] "Feedstock"             "BioProject"            "Nitrate"              
[22] "ReversePrimerSequence" "Sand"                  "CollectionYear"       
[25] "TargetGene"            "%SOM"                  "PyOM.Temp"            
[28] "NucleicAcidExtraction" "SRA"                   "NCBISampleID"         
[31] "Silt"                  "Avail_P"               "Treatment"            
[34] "g.per.kg.PyOM"         "FwdBarcode"            "ForwardPrimerName"    
[37] "Crop"                  "TSN"                   "Ammonium"             
[40] "EC"                    "RevBarcode"            "PyOM"

[1] "RevPrimerName"         "Author"                "pH"                   
 [4] "Region"                "BioSample"             "Sampling_Depth_cm"    
 [7] "Avail_K"               "ForwardPrimerSequence" "TotalN"               
[10] "CurrentLandUse"        "TotalC"                "Organism"             
[13] "SequencingMethod"      "Habitat"               "Incubation_Time_Weeks"
[16] "Org_C"                 "GeographicalLocation"  "Clay"                 
[19] "Feedstock"             "BioProject"            "Nitrate"              
[22] "ReversePrimerSequence" "Sand"                  "CollectionYear"       
[25] "TargetGene"            "%SOM"                  "PyOM.Temp"            
[28] "NucleicAcidExtraction" "SRA"                   "NCBISampleID"         
[31] "Silt"                  "Avail_P"               "Treatment"            
[34] "t.per.ha.PyOM"         "FwdBarcode"            "ForwardPrimerName"    
[37] "Crop"                  "TSN"                   "Ammonium"             
[40] "EC"                    "RevBarcode"            "PyOM"                 
[43] "SoilType"

[1] "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol"
 [7] "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol"
[13] "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol"
[19] "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol"
[25] "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol" "Fluvisol"
[31] "Fluvisol" "Fluvisol"

In [8]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('soil with biochar-mineral complex', '1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA", 'soil')
OMAdded = c('soil with organic fertilizer and biochar-mineral complex', 'soil with organic fertilizer','Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [9]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_ye)$PyOM = ifelse(sample_data(ps_ye)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_ye)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_ye)$Treatment %in% OMAdded, "OM", NA)))

In [74]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data

sample_data(ps_ye)$PyOM.pH = ifelse(sample_data(ps_ye)$PyOM %in% "PyOM", 6.98, NA)

#Add soil type as "Fluvisol"

sample_data(ps_ye)$SoilType = sample_data(ps_ye)$SoilType = "Fluvisol"

[1]   NA   NA   NA   NA   NA   NA 6.98 6.98 6.98   NA   NA   NA   NA   NA   NA
[16]   NA   NA   NA   NA   NA   NA   NA 6.98 6.98 6.98 6.98 6.98   NA   NA   NA
[31]   NA   NA

[1] "RevPrimerName"         "Author"                "pH"                   
 [4] "Region"                "BioSample"             "Sampling_Depth_cm"    
 [7] "Avail_K"               "ForwardPrimerSequence" "TotalN"               
[10] "CurrentLandUse"        "TotalC"                "Organism"             
[13] "SequencingMethod"      "Habitat"               "Incubation_Time_Weeks"
[16] "Org_C"                 "GeographicalLocation"  "Clay"                 
[19] "Feedstock"             "BioProject"            "Nitrate"              
[22] "ReversePrimerSequence" "Sand"                  "CollectionYear"       
[25] "TargetGene"            "%SOM"                  "PyOM.Temp"            
[28] "NucleicAcidExtraction" "SRA"                   "NCBISampleID"         
[31] "Silt"                  "Avail_P"               "Treatment"            
[34] "t.per.ha.PyOM"         "FwdBarcode"            "ForwardPrimerName"    
[37] "Crop"                  "TSN"                   "Ammonium"             
[40] "EC"                    "RevBarcode"            "PyOM"                 
[43] "SoilType"              "PyOM.pH"

In [75]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_ye, "ps.ye")

## Nielsen 2014

In [55]:
ps_nielsen2014 = import_biom("Nielsen2014_OTU_table/feature-table-metaD-tax_json.biom", parseFunction=parse_taxonomy_default)

In [56]:
##Fixing the tax_table object to "clean-up" the column names

x = data.frame(tax_table(ps_nielsen2014))

colnames(x) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks
x$Domain = gsub("D_0__", "", as.character(x$Domain))
x$Phylum = gsub("D_1__", "", as.character(x$Phylum))
x$Class = gsub("D_2__", "", as.character(x$Class))
x$Order = gsub("D_3__", "", as.character(x$Order))
x$Family = gsub("D_4__", "", as.character(x$Family))
x$Genus = gsub("D_5__", "", as.character(x$Genus))
x$Species = gsub("D_6__", "", as.character(x$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x=tax_table(as.matrix(x,dimnames=list(row.names(x),colnames(x))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_nielsen2014)=x
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_nielsen2014))
# Check for success

,Domain,Phylum,Class,Order,Family,Genus,Species
79ae2c2f8c966f38184481500c6b8a58,Bacteria,Firmicutes,Bacilli,Bacillales,NA,NA,NA
5386969700ee27766c23c43d036a3add,Bacteria,Verrucomicrobia,Verrucomicrobiae,Chthoniobacterales,Chthoniobacteraceae,Candidatus Udaeobacter,NA
27d37bfd668a29fa70eac0c8e621a132,Bacteria,Verrucomicrobia,Verrucomicrobiae,Chthoniobacterales,Chthoniobacteraceae,Candidatus Udaeobacter,NA
86eff6cc9a77cf0f0e31668d0a2e5618,Bacteria,Actinobacteria,Thermoleophilia,Gaiellales,uncultured,NA,NA
bc84a4db4cfa2319a4b89eb0029cf982,Bacteria,Acidobacteria,Subgroup 6,NA,NA,NA,NA
fd0fb74123bd8f51e6eaa34f69036a03,Bacteria,Actinobacteria,Actinobacteria,Frankiales,NA,NA,NA


In [59]:
#making sample data variables consistent ; fixing spelling, consitent capital letters, changing different versions 
#of Agriculture to "Agriculture"

sample_data(ps_nielsen2014)$Treatment = revalue(sample_data(ps_nielsen2014)$Treatment, c("control"="Control", "unammended"="Unamended","unamended"="Unamended"))
sample_data(ps_nielsen2014)$SoilType = revalue(sample_data(ps_nielsen2014)$SoilType, c("mollisol"="Mollisol"))
sample_data(ps_nielsen2014)$Feedstock = revalue(sample_data(ps_nielsen2014)$Feedstock, c("none"="NA"))
sample_data(ps_nielsen2014)$Habitat = revalue(sample_data(ps_nielsen2014)$Habitat, c("agriculture"="Agriculture"))
sample_data(ps_nielsen2014)$CurrentLandUse = revalue(sample_data(ps_nielsen2014)$CurrentLandUse, c("agriculture"="Agriculture", "minimim tillage agriculture field" = "Agriculture", "field" = "Agriculture"))
sample_data(ps_nielsen2014)$PyOM.Temp = revalue(sample_data(ps_nielsen2014)$PyOM.Temp, c("N/A"="NA", "0" = "NA"))
sample_data(ps_nielsen2014)$Depth = revalue(sample_data(ps_nielsen2014)$Depth, c("15cm"="15"))

The following `from` values were not present in `x`: control, unammended, unamended
The following `from` values were not present in `x`: mollisol
The following `from` values were not present in `x`: none
The following `from` values were not present in `x`: agriculture
The following `from` values were not present in `x`: agriculture, minimim tillage agriculture field, field
The following `from` values were not present in `x`: N/A, 0
The following `from` values were not present in `x`: 15cm


In [93]:
#Fixing the titles so PyOM.Temp and t.per.ha.PyOM are consistent among datasets

names(sample_data(ps_nielsen2014))[names(sample_data(ps_nielsen2014)) == "t_per_ha_PyOM"] <- "t.per.ha.PyOM"
names(sample_data(ps_nielsen2014))[names(sample_data(ps_nielsen2014)) == "PyOM_Temp"] <- "PyOM.Temp"

[1] "BioProject"            "TargetGene"            "EMBISampleID"         
 [4] "Author"                "TotalC"                "Treatment"            
 [7] "SRA"                   "CollectionYear"        "PyOM.Temp"            
[10] "pH"                    "ReversePrimerSequence" "ForwardPrimerSequence"
[13] "RevPrimerName"         "ForwardPrimerName"     "Silt"                 
[16] "Region"                "Crop"                  "SoilType"             
[19] "t.per.ha.PyOM"         "Clay"                  "SequencingMethod"     
[22] "BioSample"             "Cowell_P_mg_kg"        "Habitat"              
[25] "Sampling_Depth_cm"     "FwdBarcode"            "EC"                   
[28] "%SOM"                  "Sand"                  "Organism"             
[31] "Ammonium"              "TotalN"                "Bray_P_mg_kg"         
[34] "CurrentLandUse"        "Nitrate"               "Incubation_Time_Weeks"
[37] "RevBarcode"            "Feedstock"             "GeographicalLocation" 
[40] "NucleicAcidExtraction" "PyOM"

In [61]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('Low biochar', 'High biochar', 'soil with biochar-mineral complex', '1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA", 'soil','Fertilizer')
OMAdded = c('soil with organic fertilizer and biochar-mineral complex', 'soil with organic fertilizer','Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [50]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_nielsen2014)$PyOM = ifelse(sample_data(ps_nielsen2014)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_nielsen2014)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_nielsen2014)$Treatment %in% OMAdded, "OM", NA)))

In [76]:
#Create new column "PyOM.pH" to add the PyOM pH to the sample data

sample_data(ps_nielsen2014)$PyOM.pH = ifelse(sample_data(ps_nielsen2014)$PyOM %in% "PyOM", 6.8, NA)

[1] 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8
[20] 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8 6.8  NA  NA  NA  NA  NA
[39]  NA  NA  NA  NA

In [77]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_nielsen2014, "ps.nielsen2014")

## Xu

In [24]:
ps_Xu = import_biom("Xu_OTU_table/feature-table-metaD-tax_json.biom", parseFunction=parse_taxonomy_default)

In [25]:

x = data.frame(tax_table(ps_Xu))

colnames(x) = c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# Assigning the proper column names instead of SILVA ranks
x$Domain = gsub("D_0__", "", as.character(x$Domain))
x$Phylum = gsub("D_1__", "", as.character(x$Phylum))
x$Class = gsub("D_2__", "", as.character(x$Class))
x$Order = gsub("D_3__", "", as.character(x$Order))
x$Family = gsub("D_4__", "", as.character(x$Family))
x$Genus = gsub("D_5__", "", as.character(x$Genus))
x$Species = gsub("D_6__", "", as.character(x$Species))
# Substituting the characters we don't want with nothing in the taxonomy

x=tax_table(as.matrix(x,dimnames=list(row.names(x),colnames(x))))
# Turning it into a taxonomy table, while saving the rownames and column names

tax_table(ps_Xu)=x
# Reassigning the taxonomy table in ps_xxx to the new modified one

head(tax_table(ps_Xu))
# Check for success

,Domain,Phylum,Class,Order,Family,Genus,Species
a2f64386bf9b31f4174e4a9e338afe6a,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Nitrosomonadaceae,MND1,NA
ed67895944faf08e227f05155ee07889,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,NA,NA
1f5b5b8cda6ae40b783f5b1e8b37007b,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,TRA3-20,NA,NA
e5505c99dafc3b62ffa8c60b463b09bf,Bacteria,Proteobacteria,Alphaproteobacteria,Dongiales,Dongiaceae,Dongia,NA
e972c196dc49da1d1a67150771f2be42,Bacteria,Proteobacteria,Deltaproteobacteria,NA,NA,NA,NA
e6536447b654c8f6d13315c1eeeaa516,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Burkholderiaceae,uncultured,uncultured beta proteobacterium


In [34]:
head(sample_data(ps_Xu))

,X.SOM,TotalN,t.per.ha.PyOM,BioSample,ForwardPrimerName,RevBarcode,Crop,Avail.P,Incubation.Time.Weeks,TSN,⋯,GeographicalLocation,Avail.K,FwdBarcode,NCBISampleID,CurrentLandUse,PyOM.Temp,SequencingMethod,Org.C,Nitrate,PyOM
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
B0-X,,,0,NA,338F,,,,20,,⋯,China,,,NA,Agriculture,NA,Illumina HiSeq,16.4,,Control
B1-X,,,20,NA,338F,,,,20,,⋯,China,,,NA,Agriculture,500,Illumina HiSeq,16.4,,PyOM
B2-X,,,40,NA,338F,,,,20,,⋯,China,,,NA,Agriculture,500,Illumina HiSeq,16.4,,PyOM
B4-X,,,80,NA,338F,,,,20,,⋯,China,,,NA,Agriculture,500,Illumina HiSeq,16.4,,PyOM
B8-X,,,160,NA,338F,,,,20,,⋯,China,,,NA,Agriculture,500,Illumina HiSeq,16.4,,PyOM


In [27]:
#Fixing the titles so PyOM.Temp, t.per.ha.PyOM, and SoilType are consistent among datasets

names(sample_data(ps_Xu))[names(sample_data(ps_Xu)) == "t-per-ha-PyOM"] <- "t.per.ha.PyOM"
names(sample_data(ps_Xu))[names(sample_data(ps_Xu)) == "PyOM-Temp"] <- "PyOM.Temp"
names(sample_data(ps_Xu))[names(sample_data(ps_Xu)) == "Soil_Type"] <- "SoilType"

In [28]:
#Add soil type

sample_data(ps_Xu)$SoilType = "Cambisol"

In [30]:
#Add treatment variable

sample_data(ps_Xu)$Treatment = ifelse(sample_data(ps_Xu)$PyOM.Temp =='500', "PyOM", "NA")
                                    

In [32]:
#Create new variables to identify whether char, OM, or nothing was added to the sample

CharOnlyAdded = c('Low biochar', 'High biochar', 'soil with biochar-mineral complex', '1%BC300' ,'1%BC600','2%BC300', '2%BC600', 'biochar', 'Biochar + soil', 'Composted biochar + soil', 'High GBC' ,'Low GBC', 'PyOM',  'Soil Incubation with PyOM')
NothingAdded = c("Control","Unamended","NA", 'soil','Fertilizer')
OMAdded = c('soil with organic fertilizer and biochar-mineral complex', 'soil with organic fertilizer','Compost + biochar + soil', 'Compost + soil','Compost + biochar + soil' ,'Compost + soil','stover' ,'Straw')

In [33]:
#Create new column "PyOM" to identify whether char was added or not using the above variables

sample_data(ps_Xu)$PyOM = ifelse(sample_data(ps_Xu)$Treatment %in% CharOnlyAdded, "PyOM",
                                        ifelse(sample_data(ps_Xu)$Treatment %in% NothingAdded,"Control",
                                               ifelse(sample_data(ps_Xu)$Treatment %in% OMAdded, "OM", NA)))

In [35]:
#Save as an R object so that the phyloseq object can be called in future processing

saveRDS(ps_Xu, "ps.xu")